In [1]:
import numpy as np

from numba import njit, jit

import sys
sys.path.append('..')
from lib import *
from lib.maxent import *

In [2]:
def mcmcsampler(x0, energy, jump, nsteps=1000, nburnin=0, nsample=1, prng=None):
    """Markov chain Monte carlo sampler.

    x0: starting position (array)
    energy(x): function for calculating energy
    jump(x): function for calculating a proposed new position
    nburnin: burnin period in which states are not saved
    nsample: sample interval for saving states
    
    returns array of states
    """
    if prng is None:
        prng = np.random
    nsteps, nburnin, nsample = int(nsteps), int(nburnin), int(nsample)
    x = x0
    Ex = energy(x)
    samples = np.zeros(((nsteps-nburnin)//nsample, x0.shape[0]), dtype=np.int64)
    counter = 0
    for i in range(1, nsteps+1):
        xp = jump(x)
        Exp = energy(xp)
        if (Exp < Ex) or (prng.rand() < np.exp(-Exp+Ex)):
            x = xp
            Ex = Exp
        if (i > nburnin) and ((i-nburnin) % nsample == 0):
            samples[counter] = x
            counter += 1
    return samples

In [3]:
prng = np.random
L = 9
q = naminoacids
jump = lambda x: local_jump_jit(x, q)
x0 = prng.randint(q, size=L)
nsteps = 10000
nsample = 9

In [4]:
params = np.load('../maxent/data/Human_9.npz')
hi = params['hi']
Jij = params['Jij']
energy = lambda x: energy_potts(x, hi, Jij)

In [5]:
# unfortunately the code is buggy as the correct lenght is not always calculated
mcmcsampler(x0, energy, jump, nsteps=nsteps, nsample=nsample*3, nburnin=7)

array([[15, 17,  2, ...,  4, 14, 12],
       [ 2,  5, 13, ...,  2, 12,  5],
       [15,  8, 13, ...,  1,  7, 15],
       ...,
       [ 2,  7,  2, ..., 12,  3, 12],
       [11,  5,  8, ..., 15,  2,  0],
       [17,  9,  0, ..., 12,  2,  5]])

In [6]:
%timeit mcmcsampler(x0, energy, jump, nsteps=nsteps, nsample=nsample)

31.5 ms ± 2.35 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
%prun mcmcsampler(x0, energy, jump, nsteps=nsteps, nsample=nsample);

In [8]:
@njit
def mcmcsampler_jit(x0, energy, jump, nsteps=1000, nburnin=0, nsample=1):
    """Markov chain Monte carlo sampler.

    x0: starting position (array)
    energy(x): function for calculating energy
    jump(x): function for calculating a proposed new position
    nburnin: burnin period in which states are not saved
    nsample: sample interval for saving states
    
    returns array of states
    """
    prng = np.random
    nsteps, nburnin, nsample = int(nsteps), int(nburnin), int(nsample)
    x = x0
    Ex = energy(x)
    samples = np.zeros(((nsteps-nburnin)//nsample, x0.shape[0]), dtype=np.int64)
    counter = 0
    for i in range(1, nsteps+1):
        xp = jump(x)
        Exp = energy(xp)
        if (Exp < Ex) or (prng.rand() < np.exp(-Exp+Ex)):
            x = xp
            Ex = Exp
        if (i > nburnin) and ((i-nburnin) % nsample == 0):
            samples[counter] = x
            counter += 1
    return samples

In [9]:
@njit
def energy(x):
    return energy_potts(x, hi, Jij)

@njit
def jump(x):
    return local_jump_jit(x, q)

In [10]:
mcmcsampler_jit(x0, energy, jump, nsteps=nsteps, nsample=nsample);

In [11]:
%timeit mcmcsampler_jit(x0, energy, jump, nsteps=nsteps, nsample=nsample)

2.8 ms ± 225 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [12]:
@njit
def mcmcsampler_list(x0, energy, jump, nsteps=1000, nburnin=0, nsample=1):
    """Markov chain Monte carlo sampler.

    x0: starting position (array)
    energy(x): function for calculating energy
    jump(x): function for calculating a proposed new position
    nburnin: burnin period in which states are not saved
    nsample: sample interval for saving states
    
    returns array of states
    """
    prng = np.random
    nsteps, nburnin, nsample = int(nsteps), int(nburnin), int(nsample)
    x = x0
    Ex = energy(x)
    samples = []
    for i in range(1, nsteps+1):
        xp = jump(x)
        Exp = energy(xp)
        if (Exp < Ex) or (prng.rand() < np.exp(-Exp+Ex)):
            x = xp
            Ex = Exp
        if (i > nburnin) and (i % nsample == 0):
            samples.append(list(x))
    return np.array(samples)

In [13]:
mcmcsampler_list(x0, energy, jump, nsteps=nsteps, nsample=nsample, nburnin=10)

array([[ 0,  1,  2, ...,  5, 16,  5],
       [ 0,  1, 12, ...,  5,  8, 14],
       [14,  1, 12, ..., 16,  8, 19],
       ...,
       [ 8, 14,  7, ..., 12, 17, 13],
       [ 0, 14,  7, ..., 13,  3, 16],
       [ 0, 15,  3, ..., 13,  3, 16]])

In [14]:
%timeit mcmcsampler_list(x0, energy, jump, nsteps=nsteps, nsample=nsample)

3.22 ms ± 218 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
